<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Week 3 Assignment: Explore and cluster the neighborhoods in Toronto

The repository is a part of the IBM Data Science Capstone project. The project includes the segmentation and clustering of Neighbourhoods in Toronto using K Means Machine Learning Clustering algorithm.

In [25]:
# Assignment:

## The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
## If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
## Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Import required libraries

In [26]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


## Import and transform data from Wikipedia

Import the data from Wikepedia

In [27]:
data = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(data)
Toronto_N = tables[0]
Toronto_N.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Process the cells that have an assigned borough ignoring the ones with a borough that is Not assigned

In [28]:
Toronto_N1 = Toronto_N[(Toronto_N['Borough'] != "Not assigned")]
Toronto_N1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [29]:
Toronto_N1.shape

(103, 3)

Calculte the coordinates of the different postal codes of Toronto

In [30]:
Toronto_Coord = pd.read_csv("https://cocl.us/Geospatial_data")
Toronto_Coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge tables and build the dataframe of the postal codes of each neighborhood along with the borough name and neighborhood name

In [31]:
Toronto_N2 = Toronto_N1.merge(Toronto_Coord, on = "Postal Code")
Toronto_N2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [32]:
Toronto_N2.shape

(103, 5)

## Explore and cluster the neighborhoods in Toronto

Clustering and the plotting of the neighbourhoods of Canada which contain Toronto in their Borough

In [36]:
# Getting all the rows from the data frame which contains Toronto in their Borough

Toronto_N3 = Toronto_N2[Toronto_N2['Borough'].str.contains('Toronto',regex=False)]
Toronto_N3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Visualizing all the Neighbourhoods of the above data frame using Folium

In [37]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(Toronto_N3['Latitude'],Toronto_N3['Longitude'],Toronto_N3['Borough'],Toronto_N3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto